<a href="https://colab.research.google.com/github/NUROISEA/anime-webui-colab/blob/main/notebooks/CHANGE_THIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2>🌟 <code>Mixing Cauldron Colab</code></h2>

- 📚 [Anime Web UI Colab Wiki](https://github.com/NUROISEA/anime-webui-colab/wiki)
- 🤔 [How to use](https://github.com/NUROISEA/anime-webui-colab/wiki/Getting-started)
- 🤗 [HuggingFace Repo](https://huggingface.co/NUROISEA/mixing-cauldron)

----

[![colab count][colab-count]](https://github.com/NUROISEA/anime-webui-colab#-notebooks)
[![last update][last-update]](https://github.com/NUROISEA/anime-webui-colab/commits/main)

[colab-count]: https://img.shields.io/github/directory-file-count/NUROISEA/anime-webui-colab/notebooks?label=other%20colabs&logo=google-colab&style=flat-square&type=file
[last-update]: https://img.shields.io/github/last-commit/NUROISEA/anime-webui-colab?label=last%20update&logo=github&style=flat-square&type=file

In [ ]:
#@title 🚀 `Launch web UI`

#@markdown ### ⭐ Model selection <sup>[**[?]**](https://github.com/NUROISEA/anime-webui-colab/wiki/Selecting-a-model)</sup>
#@markdown <small> It could take 4-7 minutes to see a link to the web UI, please be patient! :) </small>
model = "exsoular.safetensors" #@param ["OrangeProBlossomPastel.safetensors", "exsoular.safetensors", "exsoulvoyant.safetensors", "floral-essence.safetensors", "florescence.safetensors", "pastel-floral-essence.safetensors"]

#@markdown <small> Select ControlNet models to use [**[?]**](https://github.com/NUROISEA/anime-webui-colab/wiki/Selecting-ControlNet-models) </small>
controlnet = "none" #@param ['none','v1.0','v1.0-diff','t2i','v1.1']
#@markdown ### 🔧 Web UI settings
#@markdown <small> This option only affects the **first** launch [**[?]**](https://github.com/NUROISEA/anime-webui-colab/wiki/First-launch-web-UI-settings) </small>
webui_version = "stable" #@param ["stable","latest","ui-redesign"]
extensions_version = "stable" #@param ["none","lite","stable","latest","experimental"]
#@markdown <small> This saves your generation to Google Drive [**[?]**](https://github.com/NUROISEA/anime-webui-colab/wiki/Saving-outputs-to-Google-Drive) </small>
outputs_to_drive = False #@param {type:'boolean'}
output_drive_folder = "AI/Generated" #@param {type:'string'}
#@markdown <small> Change tunnels if you have connection issues with `gradio` [**[?]**](https://github.com/NUROISEA/anime-webui-colab/wiki/Using-different-tunnels) </small>
tunnel = "gradio" #@param ['gradio', 'ngrok', 'cloudflared', 'localhostrun', 'remotemoe']
ngrok_token = "" #@param { type:'string' }
ngrok_region = "auto" #@param ["auto", "ap", "au", "eu", "in", "jp", "sa", "us"]

huggingface_repo = "https://huggingface.co/NUROISEA/mixing-cauldron"
model_link = f"{huggingface_repo}/resolve/main/{model}"
model_name = model_link.split("/")[-1]

vae_link = "https://huggingface.co/NoCrypt/blessed_vae/resolve/main/blessed2.vae.pt"
vae_name = vae_link.split("/")[-1]

web_ui_folder = "/content/ui"
extensions_folder = f"{web_ui_folder}/extensions"
models_folder = f"{web_ui_folder}/models/Stable-diffusion"
vae_folder = f"{web_ui_folder}/models/VAE"
embeddings_folder = f"{web_ui_folder}/embeddings"

ui_config_dictionary = {
  # refer to ui-config.json
  # "txt2img/Negative prompt/value": "(worst quality:1.4), (low quality:1.4), (monochrome:1.1)",
  # "txt2img/Sampling method/value": "DPM++ 2M Karras",
}

config_dictionary = {
  # refer to config.json
  # "CLIP_stop_at_last_layers": 1,
  # "use_scale_latent_for_hires_fix": True,
}

git_flags = "-q"
wget_flags = "-q --show-progress"

try:
  utility
except NameError:
  !wget -q https://github.com/NUROISEA/anime-webui-colab/raw/main/configs/utility.py -P /content/
  import utility
  utility.log_usage("mixing-cauldron")

utility.mount_drive(outputs_to_drive)

if not utility.has_run: 
  print('📦 Installing xformers...')
  !{utility.xformers_link}

  !{utility.install_webui(webui_version)}

  for extension_link in utility.extensions_list(extensions_version,webui_version,controlnet):
    extension_name = extension_link.split('/')[-1]
    print(f'  └ {extension_name}')
    !git clone {git_flags} {extension_link}
  # !git clone {git_flags} github.com/username/<repo> {extensions_folder}/<repo>

  for config_file in utility.configs_list():
    !wget -q {config_file} -P {web_ui_folder}/

  for embedding in utility.embeddings_list():
    !wget -q {embedding} -P {embeddings_folder}/
  # !wget {wget_flags} site.com/embedding.pt -P {embeddings_folder}/

  utility.dictionary_to_json(f"{web_ui_folder}/ui-config.json", ui_config_dictionary)
  utility.dictionary_to_json(f"{web_ui_folder}/config.json", config_dictionary)

  for patch in utility.patch_list():
    !{patch}

  utility.has_run = True

utility.output_to_gdrive(outputs_to_drive, output_drive_folder)

!{utility.download_model(model_link)}
!{utility.download_vae(vae_link)}

for controlnet_model_link in utility.controlnet_list(controlnet,webui_version,extensions_version):
  !{utility.download_controlnet(controlnet_model_link)}

%cd {web_ui_folder}

arg_list = utility.arguments(
  model=f"{models_folder}/{model_name}",
  vae=f"{vae_folder}/{vae_name}",
  tunnel=tunnel,
  ng_token=ngrok_token,
  ng_region=ngrok_region,
)

args = " ".join(arg_list)
!python launch.py {args}
#@markdown <i><small> Did something break? Please report it on [Github](https://github.com/NUROISEA/anime-webui-colab/issues) or DM me on [Reddit](https://www.reddit.com/user/CYTLNUROISEA)!

In [ ]:
#@title 📱 `Colab Mobile Keep Alive`
#@markdown <small> Run this cell (before running 🚀 `Launch web UI`) to keep the tab alive in mobile [**[?]**](https://github.com/NUROISEA/anime-webui-colab/wiki/Keeping-colab-tab-alive-in-mobile) </small>

%%html
<b>Press play on the music player to keep the tab alive before running the start cell (Uses only 13MB of data)</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

In [ ]:
#@title 💾 `Saving images`

#@markdown <small>The zip file will be visible at the files tab.</small>
from datetime import datetime
str_date = datetime.today().strftime('%Y-%m-%d-%H%M%S')
archive_name = f"outputs-{str_date}.zip"

%cd /content/ui
print("Zipping...")
!zip -qr /content/{archive_name} outputs
print(f"\033[92mZipped. You can now find {archive_name} at the files tab.\033[0m")

# ----

#@markdown <small>This copies the zip file to your Google Drive</small>
copy_to_gdrive = False #@param {type:'boolean'}
gdrive_folder = "AI/Generated" #@param { 'type': 'string' }

if copy_to_gdrive:
  utility.log_usage('zip-to-gdrive')
  from google.colab import drive

  print("Mounting to Google Drive...")
  drive.mount('/content/drive')
  if gdrive_folder == "":
    gdrive_folder = "AI/Generated"
  
  drive_folder = f"/content/drive/MyDrive/{gdrive_folder}"
  
  !mkdir -p {drive_folder}
  !cp /content/{archive_name} {drive_folder}
  print(f"\033[92mCopied to {gdrive_folder}!\033[0m")
#@markdown <small>Upload the zip file to a temporary file hosting [**[?]**](https://github.com/NUROISEA/anime-webui-colab/wiki/Uploading-to-temporary-file-hosts)</small>
upload_to_temp = False #@param {type:'boolean'}
temp_file_host = "transfer.sh" #@param ["bashupload.com", "keep.sh", "temp.sh", "transfer.sh"]

host_dictionary = {
  "bashupload.com": "https://bashupload.com",
  "keep.sh": "https://free.keep.sh",
  "temp.sh": "https://temp.sh",
  "transfer.sh": f"https://transfer.sh/{archive_name}",
}
host_link = host_dictionary[temp_file_host] # not sure if colab can do this directly at {}

if upload_to_temp:
  utility.log_usage('zip-to-temp-host')
  utility.log_usage(f'zip-temp-host-{temp_file_host}')
  print(f"Uploading to {temp_file_host}...")
  !curl -T /content/{archive_name} {host_link}  
  print("\n\033[92m^^^ This is your download link! ^^^\033[0m")
#@markdown <small>Delete colab outputs (the saved zip is unaffected)</small>
delete_local_files = False #@param {type:'boolean'}
if delete_local_files:
  utility.log_usage('local-delete-outputs')
  %cd /content/ui
  !rm -rf outputs
  print("\033[91mDeleted the outputs folder.\033[0m")